# Apply discount

This notebook modifies the json file generated by [RELOG web-base case builder](https://relog.axavier.org/casebuilder). The objective is to apply certain discount to the 40209 locations and leave the NAICS locations "as is". The discount is only applied to the opening cost.
Steps:

1. Load necessary libraries.
2. Copy and paste the case.json from the non discount folders to the respective folders
3. Load the json files these are.

## 1. Load libraries

In [1]:
import os
from datetime import datetime
import shutil
from distutils.dir_util import copy_tree
import json
import glob
import numpy as np

## 2. Copy and paste relevant files

I need to copy and paste the case.json from the non discount folders to the respective folders, these are:

* Manufacturing_BOTH_ND_1 to Manufacturing_BOTH_D_30_1
* Manufacturing_BOTHS_ND_1 to Manufacturing_BOTHS_D_30_1
* Recycling_BOTH_ND_1 to  Recycling_BOTH_D_30_1
* Recycling_BOTHS_ND_1 to Recycling_BOTHS_D_30_1

In [2]:
cwd = os.getcwd()

In [3]:
cwd

'/Users/hwikoff/Documents/RICE/3_RELOG_simulation'

In [4]:
facility_label_both = ['Manufacturing', 'Recycling']
location_label = ['BOTH', 'BOTHS']
discount_yn = ['ND', 'D']
discount_perc = ['30']
factor_label = ['1']

In [5]:
files_list = []
for faci in facility_label_both:
    for loc in location_label:
        for disc in discount_yn:
            if disc.startswith('ND'):
                for fac in factor_label:
                        files_list.append(faci+'_'+loc+'_'+disc+'_'+fac)
            else:
                for perc in discount_perc:
                    for fac in factor_label:
                        files_list.append(faci+'_'+loc+'_'+disc+'_'+perc+'_'+fac) 

In [6]:
files_list

['Manufacturing_BOTH_ND_1',
 'Manufacturing_BOTH_D_30_1',
 'Manufacturing_BOTHS_ND_1',
 'Manufacturing_BOTHS_D_30_1',
 'Recycling_BOTH_ND_1',
 'Recycling_BOTH_D_30_1',
 'Recycling_BOTHS_ND_1',
 'Recycling_BOTHS_D_30_1']

In [7]:
def copy_paste(copy, paste):
    cwd = os.getcwd()
    copy_files = os.path.join(cwd, 'simulations', copy, 'input')
    paste_files = os.path.join(cwd, 'simulations', paste, 'input')
    copy_tree(copy_files, paste_files)
    return

In [8]:
copy_paste('Manufacturing_BOTH_ND_1', 'Manufacturing_BOTH_D_30_1')
copy_paste('Manufacturing_BOTHS_ND_1', 'Manufacturing_BOTHS_D_30_1')
copy_paste('Recycling_BOTH_ND_1', 'Recycling_BOTH_D_30_1')
copy_paste('Recycling_BOTHS_ND_1', 'Recycling_BOTHS_D_30_1')

## 3. Load JSON files

We are now going to load all the case.json files corresponding to the discounted files, i.e.:

* `Manufacturing_BOTH_D_30_1`
* `Manufacturing_BOTHS_D_30_1`
* `Recycling_BOTH_D_30_1`
* `Recycling_BOTHS_D_30_1`

In [19]:
files_list_discount = [file for file in files_list if ('_D_' in file)]

In [20]:
files_list_discount

['Manufacturing_BOTH_D_30_1',
 'Manufacturing_BOTHS_D_30_1',
 'Recycling_BOTH_D_30_1',
 'Recycling_BOTHS_D_30_1']

In [21]:
for files in files_list_discount:
    discount_path = glob.glob(os.path.join(cwd, 'simulations', files, 'input','case.json'))[0]
    f = open(discount_path)
    globals()['%s' % (files)] = json.load(f) 
    f.close()

In [22]:
discount = 0.3

In [23]:
man_cap = '103954.8042'
rec_cap = '7000'
m_plant = 'PV Manufacturing Plant'
r_plant = 'PV Recycling Plant'
for file in files_list_discount:
    if 'Man' in file:
        location_file = globals()['%s' % (file)]['plants'][m_plant]['locations']
        filtered_40209_locations = [filt for filt in location_file if not (filt[-2:].isupper())]
        for filtered_loc in filtered_40209_locations:
            cost_list_before = location_file[filtered_loc]['capacities (tonne)'][man_cap]['opening cost ($)']
            apply_discount = (np.array(cost_list_before)* (1-discount)).tolist()
            location_file[filtered_loc]['capacities (tonne)'][man_cap]['opening cost ($)'] = apply_discount
        with open(os.path.join('simulations', file, 'input', 'case.json'), 'w') as outfile:
               json.dump(globals()['%s' % (file)], outfile, indent=2, separators=(',', ': '))
    else:
        location_file = globals()['%s' % (file)]['plants'][r_plant]['locations']
        filtered_40209_locations = [filt for filt in location_file if not (filt[-2:].isupper())]
        for filtered_loc in filtered_40209_locations:
            cost_list_before = location_file[filtered_loc]['capacities (tonne)'][rec_cap]['opening cost ($)']
            apply_discount = (np.array(cost_list_before)* (1-discount)).tolist()
            location_file[filtered_loc]['capacities (tonne)'][rec_cap]['opening cost ($)'] = apply_discount
        with open(os.path.join('simulations', file, 'input', 'case.json'), 'w') as outfile:
               json.dump(globals()['%s' % (file)], outfile, indent=2, separators=(',', ': '))